In [9]:
!pip install -r requirements.txt

In [14]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets
Ethiopia_Data = '1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E'
# Change 'Ethiopia_Data' to key of your own sheet in Sheets, above
x = read_sheets(Ethiopia_Data,sheet = 'Food Expenditures (2011-12)')
x['i'] = x['i'].astype(str)
x = x.set_index(['i','t','m','j']).squeeze()


# Now prices
p = read_sheets(Ethiopia_Data,sheet='Food Prices (2011-12)').set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p = p.groupby(['t','m','j','u']).median()

# Just keep metric units
p = p.xs('Kg',level="u").squeeze().unstack('j')

# Drop prices for goods we don't have expenditures for
# p = p[x.index.levels[-1]].T
p =  p.dropna()

# Household characteristics
d = read_sheets(Ethiopia_Data,sheet="Household Characteristics")
d.columns.name = 'k'

# Fill blanks with zeros
d = d.replace(np.nan,0)

# Expenditures x may have duplicate columns
x = x.T.groupby(['i','t','m','j']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)

d.set_index(['i','t','m'],inplace=True)
d = d[d.index.get_level_values('t') == '2011-12']

In [15]:
d.groupby('t').mean()

k,Females 00-03,Females 04-08,Females 09-13,Females 14-18,Females 19-30,Females 31-50,Females 51-99,Males 00-03,Males 04-08,Males 09-13,Males 14-18,Males 19-30,Males 31-50,Males 51-99,log HSize
t,,,,,,,,,,,,,,,
2011-12,0.0,0.332397,0.341843,0.272913,0.470258,0.42456,0.201174,0.0,0.341843,0.368394,0.295124,0.406178,0.378095,0.246362,1.240192


In [16]:
#filter out households in d without food expenditureslist=
i_value= y.index.get_level_values('i').unique().to_list()
d = d[d.index.get_level_values('i').isin(i_value)]

In [17]:
import cfe.regression as rgsn

result = rgsn.Regression(y=y, d=d)

In [18]:
result.predicted_expenditures()

i              t        m       j        
1010101601002  2011-12  Tigray  Banana        11.652049
                                Chat/Kat     108.598175
                                Coffee        33.892525
                                Eggs          13.791468
                                Field Pea     23.083281
                                                ...    
7210106202195  2011-12  SNNP    Salt           3.421438
                                Sorghum       30.428765
                                Sugar         18.850554
                                Teff          79.497424
                                Wheat         43.932792
Length: 47055, dtype: float64

In [19]:
y.groupby('j').count()

j
Banana            361
Barley            170
Chat/Kat          432
Cheese             48
Chick Pea         169
Coffee           2040
Eggs              217
Field Pea         655
Haricot Beans     272
Horsebeans        918
Kocho             231
Lentils           504
Linseed           172
Maize             726
Meat              905
Milk              262
Millet             79
Niger Seed         28
Onion            2572
Potato            905
Salt             3670
Sorghum           414
Sugar            1877
Teff              643
Wheat             635
Name: Expenditure, dtype: int64